## DSGE w. Feedback - Numerical Simulations

### Imports

In [1]:
%load_ext autoreload
%autoreload 2
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [2]:
%matplotlib notebook

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import ipywidgets as widgets

## Interactive Graphing

In [10]:
def timeseries(ax, df, log=True, title=''):
    
    if isinstance(df, pd.Series): 
        df = pd.DataFrame(df)

    for c in df.columns:
        ax.plot(df.loc[:,c], label=c)
    
    if df.shape[1]>1: 
        ax.legend()
    
    t = ' '.join(df.columns.to_list())
    if log:
        ax.set_yscale('log')
        t = 'log '+t
    
    if title=='':
        ax.set_title(t)
    else:
        ax.set_title(title)

In [30]:
def graph_simulation(df, params, start=0, stop=None):
    
    plt.close()
    stop = stop if stop is not None else df.shape[1]-1
    df = df.loc[start:stop,:]
    
    groups = {
        'ln Consumption & Bonds' : [df.loc[:,['c','b']], True],
        'Utility of Bonds' : [df.ft, False],
        'ln Labour' : [df.n, True],
        'ln Wages' : [df.w, True],
        'ln Income' : [df.income, True],
        'Investment %' : [df.inv, False],
        'ln Capital' : [df.k, True],
        'ln Returns' : [df.q, True],
        'News' : [df.news, False],
        'ln B/C Ratio' : [df.bc, True],
    }
    
    fig, ax = plt.subplots(ncols=1, nrows=len(list(groups.keys())))
    fig.set_size_inches(8, df.shape[1])
    axs = {}
    for i, k in enumerate(groups.keys()):
        timeseries(ax[i], *groups[k], k)
        axs[k] = ax[i]
    
    axs['ln Returns'].axhline(params['interest'], color='red', linewidth=0.5)
    
    plt.tight_layout()
    plt.show()
    
    """
    lines, l_order = [], []
        
    # Household
    l, = axs[0,0].plot(df.c[start:stop], label='c')
    lines.append(l)
    l_order.append('c')
    l, = axs[0,0].plot(df.b[start:stop], label='b')
    lines.append(l)
    l_order.append('b')
    axs[0,0].set_title('Log Consumption & Bonds')
    axs[0,0].set_yscale('log')
    axs[0,0].legend()
    l, = axs[0,1].plot(df.ft[start:stop])
    lines.append(l)
    axs[0,1].set_title('Utility of Bonds')
    
    # Labour Market
    l, = axs[1,0].plot(df.n[start:stop])
    lines.append(l)
    l_order.append('n')
    axs[1,0].set_title('Log Labour')
    axs[1,0].set_yscale('log')
    l, = axs[1,1].plot(df.w[start:stop])
    lines.append(l)
    l_order.append('w')
    axs[1,1].set_title('Log Wages')
    axs[1,1].set_yscale('log')
    
    # Investment Rate
    l, = axs[2,0].plot(df.income[start:stop])
    lines.append(l)
    l_order.append('income')
    axs[2,0].set_title('Log Income')
    axs[2,0].set_yscale('log')
    l, = axs[2,1].plot(df.inv[start:stop], label = r'Investment [%]')
    lines.append(l)
    l_order.append('inv')
    axs[2,1].set_title('Investment [% of income]')
    
    # Capital Market
    l, = axs[3,0].plot(df.k[start:stop], label='k')
    lines.append(l)
    l_order.append('k')
    axs[3,0].set_title('Log Capital')
    axs[3,0].set_yscale('log')
    #axs[3,0].plot(investment, color='red', label='inv')
    axs[3,0].legend(ncol=2)
    
    l, = axs[3,1].plot(df.q[start:stop], label = 'Return to Capital')
    lines.append(l)
    l_order.append('q')
    axs[3,1].set_title('Log Return to Capital')
    axs[3,1].axhline(params['interest'], color='red', linewidth=0.5, label='Interests rate')
    axs[3,1].set_yscale('log')
    axs[3,1].legend()
    
    # News
    l, = axs[4,0].plot(df.news[start:stop])
    lines.append(l)
    l_order.append('news')
    axs[4,0].set_title('News')
    
    l, = axs[4,1].plot(df.bc[start:stop])
    lines.append(l)
    l_order.append('bc')
    axs[4,1].axhline(0, color='red')
    axs[4,1].set_yscale('log')
    axs[4,1].set_title('Bond/Consumption Ratio')
    
    fig.tight_layout()
    plt.show()
    return fig, axs, lines, l_order
    """

## Simulation mechanics

In [14]:
def lhs(cons: float, z: float, k:float, alpha: float, gamma: float = 1, r: float = 0) -> float:
    pt1 = 2 * gamma * (1+r) / (1 - alpha)
    pt2 = (cons / (z * k**alpha)) ** (2 / (1-alpha))
    return pt1 * pt2

def rhs(cons: float, gti: float, ft: float) -> float:
    return 1 - (ft * cons) / (gti - cons)

def bisection_generalCB(z:float, income:float, k:float, gt_:float, ft_:float, p:dict, err:float=1e-5)->float:
    
    # Define the left and right hand sides of the equations
    diff = lambda c: rhs(c, gt_ * income, ft_) - lhs(c, z, k, p['alpha'], p['gamma'], p['interest'])

    # Initial guess at the next options for
    max_val = gt_ * income / (1+ft_)
    x = [0, max_val / 2, max_val]
    abs_lst = [abs(diff(i)) for i in x[:2]]

    while min(abs_lst) >= err:
        test = np.sign([diff(i) for i in x])

        if test[0] == test[1]:
            x = [x[1], (x[1] + x[2]) / 2, x[2]]
        elif test[1] == test[2]:
            x = [x[0], (x[0] + x[1]) / 2, x[1]]

        abs_lst = [abs(diff(i)) for i in x[:2]]

    return x[np.argmin(abs_lst)]

In [15]:
def step(t: float, x: np.ndarray, p: dict):
    # Starting variables
    z_, c_, n_, b_, w_, k_, q_, gt_, ft_, news_, inc_, xiz_, xin_ = x
        
    # Random technology process
    rand = np.random.normal(0, p['sigmaZ'])
    xiz = p['etaZ'] * xiz_ + np.sqrt(1 - p['etaZ'] ** 2) * rand
    z = p['zbar'] * np.exp(xiz)
        
    # Income and Investment
    income = (w_ * n_ + b_  + q_ * k_) / (1 + p['inflation'])
    
    # Capital Markets
    k = (1 - p['depreciation']) * k_ + income * (1 - gt_)

    # Household decision
    c = bisection_generalCB(z, income, k, gt_, ft_, p)
    n = (c ** 2) / (4 * k * (z ** 2))
    b = (gt_ * income - c) * (1 + p['interest'])
    
    # Firm decisions
    w = (1-p['alpha']) * z * (k / n) ** p['alpha']
    q = p['alpha'] * z * (n / k) ** (1 - p['alpha'])
    #CES
    #temp = (p['alpha'] * k ** p['rho'] + (1 - p['alpha']) * n ** p['rho']) ** ((1 / p['rho']) - 1)
    #w = (1 - p['alpha']) * z * temp * (n ** (p['rho'] - 1))
    #q = p['alpha'] * z * temp * (k ** (p['rho'] - 1))

    # News
    xin = np.random.normal(0, p['sigmaN'])
    info = p['n_cons']*(c/c_ - 1)
    news = np.tanh(p['n_theta'] * (p['n_persistence'] * news_ + (1 - p['n_persistence']) * info + xin))
    
    if t > 300 and t < 400:
        if p['shock'] == -1:
            news = -1
        elif p['shock'] == 1:
            news = 1
    
    # Household modifiers
    gt = 0.5 * (p['g_max'] + p['g_min'] - news * (p['g_max'] - p['g_min']))
    ft = 0.5 * (p['f_max'] + p['f_min'] - news * (p['f_max'] - p['f_min']))
    
    return z, c, n, b, w, k, q, gt, ft, news, income, xiz, xin

In [16]:
def simulate(start: np.ndarray, p: dict, t_end: float = 1e3):
    x = np.empty((int(t_end), len(start)))
    x[0, :] = start
    for t in range(1, int(t_end)):
        x[t, :] = step(t, x[t - 1, :], p)
    cols = ['z', 'c', 'n', 'b', 'w', 'k', 'q', 'gt', 'ft', 'news', 'income', 'xiz', 'xin']
    df = pd.DataFrame(x, columns=cols)
    df.loc[:,'inv'] = 100*(1-df.loc[:,'gt'])
    df.loc[:,'bc'] = df.b / df.c
    return df

In [17]:
def sim_and_plot(etaZ=0.1, sigmaZ=0.8, zbar=1, sigmaN=0.2, inflation=0.01, interest=0.01, 
                 depreciation=0.1,g_min=0.95, g_max=1-1e-4,f_min=0.0, f_max=1000, n_persistence=0.8, 
                 n_theta=1.6, n_cons=0.1,gamma=1, alpha=0.5, shock=0):

    df = simulate(start, locals(), T)
    graph_simulation(df, locals(), start=view[0], stop=view[1])
    plt.show()

----
## Interactive Analysis

In [31]:
param_limits = dict(
    etaZ=dict(value=0.2, min=0, max=1, step=0.1, description=r'$\eta_z$'),
    sigmaZ=dict(value=0.8, min=0, max=2, step=0.1, description=r'$\sigma_z$'),
    zbar=dict(value=1.0, min=0, max=10, step=1.0, description=r'$\bar{z}$'),
    sigmaN=dict(value=0.2, min=0, max=1, step=0.1, description=r'$\sigma_\xi$'),
    inflation=dict(value=0.01, min=0, max=0.1, step=0.01, description=r'$\pi$'),
    interest=dict(value=0.01, min=0, max=0.1, step=0.01, description=r'$r$'),
    depreciation=dict(value=0.01, min=0, max=0.1, step=0.01, description=r'$\delta$'),
    g_min=dict(value=0.8, min=0,  max=1,  step=0.1, description=r'$\varrho_\min$'),
    g_max=dict(value=0.99, min=0,  max=1,  step=0.1, description=r'$\varrho_\max$'),
    f_min=dict(value=1.0, min=0,  max=10,  step=1, description=r'$\varphi_\min$'),
    f_max=dict(value=100, min=0,  max=1000,  step=10, description=r'$\varphi_\max$'),
    n_cons=dict(value=1, min=0,  max=10,  step=0.1, description=r'$\theta_c$'),
    n_theta=dict(value=1, min=0,  max=10,  step=0.1, description=r'$\theta_\nu$'),
    n_persistence=dict(value=0.8, min=0,  max=1,  step=0.05, description=r'$\rho_\nu$'),
    gamma=dict(value=1.0, min=0, max=10,  step=1.0, description=r'$\gamma$'),
    alpha=dict(value=0.5, min=0, max=1,  step=0.1, description=r'$\alpha$'),
    shock=dict(value=0.0, min=-1, max=1,  step=1.0, description=r'Shock'),
)

start = dict(z=1, c=100, n=100, b=100, w=1, k=100, q=0.1, gt=0.8, ft=.5, income=0, news=1, xiz=0, xin=0)
start = np.array([v for _,v in start.items()])

T = 1e3
np.random.seed(40)
view = (1, int(1e3))

widget_kwargs = dict(disabled=False, continuous_update=False, orientation='horizontal',
                     readout=True, readout_format='.2f')
    
var_widgets = {k:widgets.FloatSlider(**p, **widget_kwargs) for k, p in param_limits.items()}

out = widgets.interactive_output(sim_and_plot, var_widgets)
temp = [v for _,v in var_widgets.items()]
#widgets.VBox([widgets.HBox(temp), out])
widgets.AppLayout(header=None,
          left_sidebar=widgets.VBox(temp),
          center=out,
          right_sidebar=None,
          footer=None,
          grid_gap='10px',
          justify_items='left',
          align_items='center')

AppLayout(children=(VBox(children=(FloatSlider(value=0.2, continuous_update=False, description='$\\eta_z$', ma…